In [1]:
!pip install azure-ai-textanalytics==5.3.0

     ---------------------------------------- 0.0/82.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/82.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/82.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/82.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/82.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/82.8 kB ? eta -:--:--
     ------------------- -------------------- 41.0/82.8 kB ? eta -:--:--
     ---------------------------------------- 82.8/82.8 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/298.6 kB ? eta -:--:--
   --------- ------------------------------ 71.7/298.6 kB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 143.4/298.6 kB 1.4 MB/s eta 0:00:01
   --------------------------- ------------ 204.8/298.6 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------  297.0/298.6 kB 1.7 MB/s eta 0:00:01
   ----------------------

In [2]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import (
    TextAnalyticsClient,
    ExtractiveSummaryAction
) 
import pandas as pd

In [3]:
print("Cargar variables de entorno desde archivo .env")
load_dotenv("env.txt",override=True)

Cargar variables de entorno desde archivo .env


True

In [15]:

key = os.environ.get('LANGUAGE_KEY')
endpoint = os.environ.get('LANGUAGE_ENDPOINT')

# Autenticarse
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Ejemplo para resumir texto
def sample_extractive_summarization(client, documents):
    document = documents

    poller = client.begin_analyze_actions(
        document,
        actions=[
            ExtractiveSummaryAction(max_sentence_count=1)
        ],
    )

    document_results = poller.result()
    for result in document_results:
        extract_summary_result = result[0]  # first document, first result
        if extract_summary_result.is_error:
            print("Error: '{}' - Mensaje: '{}'".format(
                extract_summary_result.code, extract_summary_result.message
            ))
        else:
            print("Resumen: \n{}".format(
                " ".join([sentence.text for sentence in extract_summary_result.sentences]))
            )
# Example function for recognizing entities from text
def entity_recognition_example(client, documents):
    try:
        result = client.recognize_entities(documents = documents)[0]

        print("Named Entities:\n")
        return result.entities
    
    except Exception as err:
        print("Encountered exception. {}".format(err))



In [21]:
document = "Elon Reeve Musk (Pretoria, 28 de junio de 1971), \
    conocido como Elon Musk, es un empresario, inversor y magnate sudafricano \
    que también posee las nacionalidades canadiense y estadounidense. Es el fundador, \
X Corp."

In [14]:
sample_extractive_summarization(client,document)

Resumen: 
Elon Reeve Musk (Pretoria, 28 de junio de 1971),     conocido como Elon Musk, es un empresario, inversor y magnate sudafricano     que también posee las nacionalidades canadiense y estadounidense. Es el fundador,     consejero delegado e ingeniero jefe de SpaceX;


In [22]:
entities = entity_recognition_example(client,list(document))

Encountered exception. (InvalidDocumentBatch) Batch request contains too many records. Max 5 records are permitted.
Code: InvalidDocumentBatch
Message: Batch request contains too many records. Max 5 records are permitted.


In [9]:
data_list = []

for entity in entities:
    data_list.append([entity.text, entity.category, entity.confidence_score])
    
columns = ['Name','Category','Confidence Score']
df = pd.DataFrame(data_list, columns=columns)
df

,Name,Category,Confidence Score
0,Elon Reeve Musk,Person,1.00
1,Pretoria,Location,1.00
2,28 de junio de 1971,DateTime,1.00
3,Elon Musk,Person,1.00
4,empresario,PersonType,0.99
5,inversor,PersonType,0.97
6,magnate,PersonType,0.96
7,sudafricano,PersonType,0.64
8,fundador,PersonType,0.98
9,consejero delegado,PersonType,0.96


In [18]:
list(df[df['Category'] == 'Person'].Name)

['Elon Reeve Musk', 'Elon Musk', 'inversor ángel']